In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import pandas as pd
import cvxopt
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

# Multiplicadores de Lagrange 
$\frac{\partial f(x)}{ \partial x} = λ \frac{\partial g(  x)}{ \partial x}$

Hallar los valores de $λ_i$ para cada elemento de entrenamiento $X_i$. 

El código ***GetLambda***  debe retornar un vector al cual denominaremos lambda, de modo que
  $lambda[i]$ será $0$, si el elemento $X[i]$ no tiene intercesión con ninguna de las rectas
  $XW^t + b >=1$ o $XW^t + b >=0$

Nota: X es una matriz, de modo que $X_i$ será un vector de dimensión $K$ que representa el i-esimo objeto o punto $k$ dimensional y $X_ij$ 



- **Nota: Puede buscar en internet la forma de como hallar lambda.**

In [ ]:
def gaussian_kernel(x, y, sigma=5.0):
  return np.exp(-(np.linalg.norm(x-y))**2 / (2 * (sigma ** 2)))

def generate_k(X):
  n_samples = X.shape[0]
  K = np.zeros((n_samples, n_samples))
  for i in range(n_samples):
    for j in range(n_samples):
      #K[i,j] = gaussian_kernel(np.array(X.iloc[i].to_list()), np.array(X.iloc[j].to_list()))
      K[i, j] = gaussian_kernel(X[i], X[j])
  return K

def GetLambda(X,Y,C=0.1):   
  # write your code here 
  K = generate_k(X)
  n_samples = X.shape[0]
  
  np.outer(Y, Y)
  Y = Y.astype(np.double)
  P = cvxopt.matrix(np.outer(Y,Y) * K)
  q = cvxopt.matrix(np.ones(n_samples) * -1)
  A = cvxopt.matrix(Y, (1,n_samples))
  b = cvxopt.matrix(0.0)

  if C is None:
    G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
    h = cvxopt.matrix(np.zeros(n_samples))
  else:
    tmp1 = np.diag(np.ones(n_samples) * -1)
    tmp2 = np.identity(n_samples)
    G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
    tmp1 = np.zeros(n_samples)
    tmp2 = np.ones(n_samples) * C
    h = cvxopt.matrix(np.hstack((tmp1, tmp2)))

  print(len(A))
  solution = cvxopt.solvers.qp(P, q, G, h, A, b)
  return np.ravel(solution['x'])

## 2 Cálculo de los pesos W
$W_j = \sum_{i=0}^n \lambda_iy_ix_{ij}$  

Donde: λ_i es el i-esimo multiplicador de lagrange, W_j es el W-esimo peso y x_{ij} es el valor de la característica $j$ del objeto de entrenamiento $i-esimo$ y $y_i$ es la salida esperada (1 o - 1) del objeto $i$.

Recuerde la sumatoria solo recorre todos los elementos para los cuales el valor del multiplicador de lagrange $λ_i$ es diferente de 0.

In [ ]:
def Get_W(X,Y, lambda):
# write your code here

## Cálculo de b

XW^t + b = 0 

$b = - \frac{1}{n}∑_{i=0}^n X_iW^t$

Donde $X_i$ es un vector $k$ dimensional y representa el objeto $i-esimo$ de entrenamiento y $k$ el número de características del objeto.

In [ ]:
def Get_b(X,W):
  # write your code here

## Etapa de Testing

Para esta estapa solo se debe calcular

$f(X_j) = X_jW^t + b$

Pero dado que ya hemos calculado el valor de los parámetros $W$ y $b$, entonces remplazando tenemos

$f(X_j) = \sum_{i=0}^n \lambda_iy_i<X_{i},X_{j}> + b$

Donde: $X_i$ i-esimo  es el vector de entrenamiento y $X_j$ es el nuevo vector que pasa por el modelo para su predicciòn predecir la clase (1 o -1).

Finalmente para saber a que clase pertenece el nuevo vector $X_j$ vasta con verificar el signo de f(X_j). 

  - **If $f(X_j) >=0$ then $Y_j$ = 1 else $Y_j = -1$**

In [ ]:
def Test(X,W,b):
  # write your code here

Base de Datos para Las pruebas:
[Download](https://gist.github.com/netj/8836201)

En esta base de datos existen 3 clases, solo utilize dos clases para hacer las pruebas.

- Separe el dataset en 70% para entrenar y 30% para hacer las pruebas

- Añada un valor 1 para la primera clase  y  -1 para la segunda clase.

- En la etapa de test, encuentre el número de elementos correctamente clasificados y el número de elementos incorrectamente clasificados para cada clase.

- Cree una matriz de confusión el cual nos mostrará la eficiencia del método.